In [ ]:
from imbd.data import DataLoader, DataPreprocessor
from imbd.training import ModelTrainer
import numpy as np
import pandas as pd

In [ ]:
loader = DataLoader()

In [ ]:
df = loader.build(data_type='label_20')

In [ ]:
preprocessor = DataPreprocessor()

In [ ]:
train_features = df.drop(loader.labels, axis=1)
train_labels = df[loader.labels]

In [ ]:
train_features = preprocessor(train_features)

In [ ]:
param_grid = {
"estimator__n_estimators": [1000],
"estimator__max_depth": [2, 5, 10],
# "estimator__alpha": [0, 0.1],
# "estimator__lambda": [1, 0.5],
"estimator__subsample": [1, 0.5],
# "estimator__gamma": [0, 2],
}

trainer = ModelTrainer(param_grid=param_grid)
trainer.train(train_features, train_labels)

In [ ]:
trainer.training_result

In [ ]:
pred = trainer.predict(train_features)
pred = np.round(pred, 2)

In [ ]:
res = np.abs(pred - train_labels)
res['mean_abs_diff'] = np.mean(res, axis=1)
res['outlier'] = train_features['outlier']


In [ ]:
res.sort_values('mean_abs_diff', ascending=False).to_csv('output.csv', index=False)

In [ ]:
import plotly.express as px

In [ ]:
columns = train_features.filter(regex='Input_C_[0-9]+').columns
# columns = train_features.filter(regex='Input_A3_[0-9]+').columns
target = pd.concat([train_features[columns[:20]], train_features['outlier'], res['mean_abs_diff']], axis=1)
target.columns

In [ ]:
px.parallel_coordinates(target, color='mean_abs_diff', color_continuous_scale=px.colors.diverging.Tealrose)

In [ ]:
pred_df = pd.DataFrame(pred, columns=train_labels.columns)

In [ ]:
test = pd.concat([pred_df['Input_A1_020'], train_labels['Input_A1_020']], axis=1)

In [ ]:
res.sort_values('mean_abs_diff', ascending=False)